In [1]:
import pandas as pd
import json

df_csv = pd.read_csv('UNSPSC_English.csv')
df_xlsx = pd.read_excel('Avalara_goods_and_services.xlsx')

/var/folders/ly/fjs87q054hz8c44_w05zm2kh0000gn/T/ipykernel_6348/4190874567.py:4: DtypeWarning: Columns (1,2,5,8,9,12,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_csv = pd.read_csv('UNSPSC_English.csv')


In [2]:
df_xlsx = df_xlsx.rename(columns={'Unnamed: 0':'AvaTax System Tax Code', 'AvaTax System Tax Codes':'AvaTax System Tax Code Description',
                                 'Unnamed: 2':'Additional AvaTax System Tax Code Information'})
df_xlsx = df_xlsx[2:200]
df_xlsx

,AvaTax System Tax Code,AvaTax System Tax Code Description,Additional AvaTax System Tax Code Information
2,D0000000,Digital goods,Digital goods are generally viewed as download...
3,D9999999,Temporary unmapped digital goods sku - taxable...,A digital good that has not been mapped to a s...
4,DA010000,Digital goods: advertising material (digital o...,"Tangible personal property (i.e. designs, artw..."
5,DA030000,Digital goods: art (digital only -- no physica...,Canned artwork that is transferred to the cust...
6,DA040000,Digital goods: audio (digital only -- no physi...,Audio (i.e. music or spoken word) that is tran...
...,...,...,...
195,PA020662,Food producing woody plants,Includes: blueberry and raspberry bush / apple...
196,PA020664,Insecticides / fungicides for food producing p...,NaN
197,PA020665,Flower seeds,Excludes flower bulbs.
198,PA020668,Dual-purpose fertilizer: non-food producing pl...,Primarily fertilizes and kills weeds.


In [3]:
# import nltk and download resources
import nltk
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dizhang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/dizhang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
# parse the stop words and special characters out
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))
stop_words.add(':')
stop_words.add('--')
stop_words.add('-')
stop_words.add('(')
stop_words.add(')')
stop_words.add('/')
stop_words.add('*')
stop_words.add(';')
stop_words.add('.')
stop_words.add('i.e')
stop_words.add('A')
stop_words.add('a')
stop_words.add(',')

In [5]:
def createParsedColumns(file, col1, col2, start_index, new_col):
    for index1, description1 in enumerate(file[col1], start_index):
        if not pd.isna(description1):
            word_tokens1 = word_tokenize(description1)

            filtered_sentence1 = []
            for w1 in word_tokens1:
                if w1 not in stop_words:
                    filtered_sentence1.append(w1.lower())

            # remove empty strings
            while "" in filtered_sentence1:
                filtered_sentence1.remove("")

            # updating record in col1
            file.at[index1, col1]= ','.join(filtered_sentence1)
        else:
            file.at[index1, col1]= ''

    for index2, description2 in enumerate(file[col2], start_index):
        if not pd.isna(description2):
            word_tokens2 = word_tokenize(description2)

            filtered_sentence2 = []
            for w2 in word_tokens2:
                if w2 not in stop_words:
                    filtered_sentence2.append(w2.lower())

            # remove empty strings
            while "" in filtered_sentence2:
                filtered_sentence2.remove("")

            # updating record in col2
            file.at[index2, col2]= ','.join(filtered_sentence2)
        else:
            file.at[index2, col2]= ''

    # create 3rd column
    new_column_series = []
    for index3, description3 in enumerate(file[col2], start_index):
        temp1 = file[col1].loc[index3].split(',')

        if not pd.isna(description3):
            temp2 = file[col2].loc[index3].split(',')
            temp3 = temp1 + temp2
            temp3 = set(temp3)

            if '' in temp3:
                temp3.remove('')

            new_column_series.append(list(temp3))
        else:
            new_column_series.append(list(temp1))

    # insert new column
    file.loc[:,new_col] = new_column_series

In [6]:
column_name = 'AvaTax System Tax Code Description'
column_name_2 = 'Additional AvaTax System Tax Code Information'
new_col_name = 'tax_parsed'
createParsedColumns(df_xlsx, column_name, column_name_2, 2, new_col_name)
df_xlsx

,AvaTax System Tax Code,AvaTax System Tax Code Description,Additional AvaTax System Tax Code Information,tax_parsed
2,D0000000,"digital,goods","digital,goods,generally,viewed,downloadable,it...","[software, items, generally, pdfs, artwork, we..."
3,D9999999,"temporary,unmapped,digital,goods,sku,taxable,d...","digital,good,mapped,specific,digital,good,tax,...","[limited, temporary, reviewed, default, adequa..."
4,DA010000,"digital,goods,advertising,material,digital,phy...","tangible,personal,property,designs,artwork,pho...","[tangible, etc, personal, artwork, property, s..."
5,DA030000,"digital,goods,art,digital,physical,media","canned,artwork,transferred,customer,electronic...","[canned, digital, goods, media, electronic, ar..."
6,DA040000,"digital,goods,audio,digital,physical,media","audio,music,spoken,word,transferred,customer,e...","[digital, goods, media, spoken, music, electro..."
...,...,...,...,...
195,PA020662,"food,producing,woody,plants","includes,blueberry,raspberry,bush,apple,pear,t...","[ornamental, bush, apple, vines, raspberry, wo..."
196,PA020664,"insecticides,fungicides,food,producing,plants",,"[fungicides, plants, producing, food, insectic..."
197,PA020665,"flower,seeds","excludes,flower,bulbs","[flower, excludes, bulbs, seeds]"
198,PA020668,"dual-purpose,fertilizer,non-food,producing,pla...","primarily,fertilizes,kills,weeds","[fertilizes, non-food, weeds, primarily, insec..."


In [7]:
tc_hash_map = {}

new_col_name = 'tax_parsed'

for index, tax_code in enumerate(df_xlsx['AvaTax System Tax Code'], 2):

    # if tax_code = NaN
    if isinstance(tax_code, float):
        prefix = df_xlsx["AvaTax System Tax Code"].loc[index + 1]
        level_1 = prefix[0:2]
        level_2 = prefix[2:4]
        level_3 = "0000"
        df_xlsx["AvaTax System Tax Code"].loc[index] = prefix[0:4]+"0000"
    else:
        level_1 = tax_code[0:2]
        level_2 = tax_code[2:4]
        level_3 = tax_code[4:8]
    

    if level_1 not in tc_hash_map:
        temp_dict0 = {level_3: df_xlsx[new_col_name].loc[index]}
        temp_dict = {level_2: [df_xlsx[new_col_name].loc[index], temp_dict0]}
        tc_hash_map[level_1] = [df_xlsx[new_col_name].loc[index], temp_dict]
    elif level_2 not in tc_hash_map[level_1][1]:
        temp_dict0 = {level_3: df_xlsx[new_col_name].loc[index]}
        tc_hash_map[level_1][1][level_2] = [df_xlsx[new_col_name].loc[index], temp_dict0]
    elif level_3 not in tc_hash_map[level_1][1][level_2][1]:
        tc_hash_map[level_1][1][level_2][1][level_3] = df_xlsx[new_col_name].loc[index]

print(json.dumps(tc_hash_map, sort_keys=True, indent=2))

{
  "D0": [
    [
      "software",
      "items",
      "generally",
      "pdfs",
      "artwork",
      "websites",
      "viewed",
      "include",
      "computer",
      "sold",
      "zip",
      "digital",
      "music",
      "photographs",
      "electronically",
      "examples",
      "downloadable",
      "otherwise",
      "movies",
      "goods",
      "e-books",
      "transferred",
      "would",
      "files"
    ],
    {
      "00": [
        [
          "software",
          "items",
          "generally",
          "pdfs",
          "artwork",
          "websites",
          "viewed",
          "include",
          "computer",
          "sold",
          "zip",
          "digital",
          "music",
          "photographs",
          "electronically",
          "examples",
          "downloadable",
          "otherwise",
          "movies",
          "goods",
          "e-books",
          "transferred",
          "would",
          "files"
        ],
        {
  

In [8]:
tc_table_lvl_1 = {}
tc_table_lvl_2 = {}
tc_table_lvl_3 = {}

new_col_name = 'tax_parsed'

for index, tax_code in enumerate(df_xlsx['AvaTax System Tax Code'], 2):
    # if tax_code = NaN
    if isinstance(tax_code, float):
        prefix = df_xlsx["AvaTax System Tax Code"].loc[index + 1]
        level_1 = prefix[0:2]
        level_2 = prefix[2:4]
        level_3 = "0000"
        df_xlsx["AvaTax System Tax Code"].loc[index] = prefix[0:4]+"0000"
    else:
        level_1 = tax_code[0:2]
        level_2 = tax_code[2:4]
        level_3 = tax_code[4:8]


    if level_1 not in tc_table_lvl_1:
        tc_table_lvl_1[level_1] = df_xlsx[new_col_name].loc[index]

    if level_1 + level_2 not in tc_table_lvl_2:
        tc_table_lvl_2[level_1+level_2] = df_xlsx[new_col_name].loc[index]

    if level_1 + level_2 + level_3 not in tc_table_lvl_3:
        tc_table_lvl_3[level_1+level_2+level_3] = df_xlsx[new_col_name].loc[index]

print(json.dumps(tc_table_lvl_1, sort_keys=True, indent=2))
print(json.dumps(tc_table_lvl_2, sort_keys=True, indent=2))
print(json.dumps(tc_table_lvl_3, sort_keys=True, indent=2))

{
  "D0": [
    "software",
    "items",
    "generally",
    "pdfs",
    "artwork",
    "websites",
    "viewed",
    "include",
    "computer",
    "sold",
    "zip",
    "digital",
    "music",
    "photographs",
    "electronically",
    "examples",
    "downloadable",
    "otherwise",
    "movies",
    "goods",
    "e-books",
    "transferred",
    "would",
    "files"
  ],
  "D9": [
    "limited",
    "temporary",
    "reviewed",
    "default",
    "adequately",
    "digital",
    "good",
    "impose",
    "unless",
    "exemption",
    "cases",
    "purpose",
    "use",
    "using",
    "assigned",
    "goods",
    "unmapped",
    "tax",
    "base",
    "applicable",
    "accurately",
    "getting",
    "system",
    "sku",
    "mapped",
    "taxable",
    "code",
    "entity",
    "specific"
  ],
  "DA": [
    "tangible",
    "etc",
    "personal",
    "artwork",
    "property",
    "services",
    "digital",
    "advertising",
    "associated",
    "goods",
    "means",
    "d

In [9]:
# Convert row to header and remove the row
df_csv = df_csv.rename(columns=df_csv.iloc[8])
df_csv = df_csv.loc[9:2000]
df_csv = df_csv.drop(columns = ['Version', 'Key', 'Column1', 'Synonym', 'Acronym'])
df_csv

,Segment,Segment Title,Segment Definition,Family,Family Title,Family Definition,Class,Class Title,Class Definition,Commodity,Commodity Title,Definition
9,10000000,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,10000000,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",10100000,Live animals,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,10000000,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",10100000,Live animals,NaN,10101500,Livestock,NaN,NaN,NaN,NaN
12,10000000,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",10100000,Live animals,NaN,10101500,Livestock,NaN,10101501,Cats,NaN
13,10000000,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",10100000,Live animals,NaN,10101500,Livestock,NaN,10101502,Dogs,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1996,10000000,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",10210000,Live plants of high species or variety count f...,Live plants including the root ball of flower ...,10216000,Live ornamental peppers,Live plants including the root ball of ornamen...,NaN,NaN,NaN
1997,10000000,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",10210000,Live plants of high species or variety count f...,Live plants including the root ball of flower ...,10216000,Live ornamental peppers,Live plants including the root ball of ornamen...,10216001,Live ornamental chili pepper,Live plant including the root ball of the orna...
1998,10000000,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",10210000,Live plants of high species or variety count f...,Live plants including the root ball of flower ...,10216000,Live ornamental peppers,Live plants including the root ball of ornamen...,10216002,Live ornamental mixed pepper,Live plant including the root ball of the orna...
1999,10000000,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",10210000,Live plants of high species or variety count f...,Live plants including the root ball of flower ...,10216000,Live ornamental peppers,Live plants including the root ball of ornamen...,10216003,Live ornamental orange pepper,Live plant including the root ball of the orna...


In [10]:
column_name = 'Segment Title'
column_name_2 = 'Segment Definition'
column_name_3 = 'Family Title'
column_name_4 = 'Family Definition'
column_name_5 = 'Class Title'
column_name_6 = 'Class Definition'
column_name_7 = 'Commodity Title'
column_name_8 = 'Definition'
createParsedColumns(df_csv, column_name, column_name_2, 9, 'segment_parsed')
createParsedColumns(df_csv, column_name_3, column_name_4, 9, 'family_parsed')
createParsedColumns(df_csv, column_name_5, column_name_6, 9, 'class_parsed')
createParsedColumns(df_csv, column_name_7, column_name_8, 9, 'commodity_parsed')
df_csv

,Segment,Segment Title,Segment Definition,Family,Family Title,Family Definition,Class,Class Title,Class Definition,Commodity,Commodity Title,Definition,segment_parsed,family_parsed,class_parsed,commodity_parsed
9,10000000,"live,plant,animal,material,accessories,supplies","this,segment,includes,live,wild,domesticated,s...",NaN,,,NaN,,,NaN,,,"[materials, care, live, accessories, animals, ...",[],[],[]
10,10000000,"live,plant,animal,material,accessories,supplies","this,segment,includes,live,wild,domesticated,s...",10100000,"live,animals",,NaN,,,NaN,,,"[materials, care, live, accessories, animals, ...","[animals, live]",[],[]
11,10000000,"live,plant,animal,material,accessories,supplies","this,segment,includes,live,wild,domesticated,s...",10100000,"live,animals",,10101500,livestock,,NaN,,,"[materials, care, live, accessories, animals, ...","[animals, live]",[livestock],[]
12,10000000,"live,plant,animal,material,accessories,supplies","this,segment,includes,live,wild,domesticated,s...",10100000,"live,animals",,10101500,livestock,,10101501,cats,,"[materials, care, live, accessories, animals, ...","[animals, live]",[livestock],[cats]
13,10000000,"live,plant,animal,material,accessories,supplies","this,segment,includes,live,wild,domesticated,s...",10100000,"live,animals",,10101500,livestock,,10101502,dogs,,"[materials, care, live, accessories, animals, ...","[animals, live]",[livestock],[dogs]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996,10000000,"live,plant,animal,material,accessories,supplies","this,segment,includes,live,wild,domesticated,s...",10210000,"live,plants,high,species,variety,count,flowers","live,plants,including,root,ball,flower,types,5...",10216000,"live,ornamental,peppers","live,plants,including,root,ball,ornamental,pep...",NaN,,,"[materials, care, live, accessories, animals, ...","[unspsc, root, species, high, variety, live, f...","[pepper, peppers, ornamental, root, live, flow...",[]
1997,10000000,"live,plant,animal,material,accessories,supplies","this,segment,includes,live,wild,domesticated,s...",10210000,"live,plants,high,species,variety,count,flowers","live,plants,including,root,ball,flower,types,5...",10216000,"live,ornamental,peppers","live,plants,including,root,ball,ornamental,pep...",10216001,"live,ornamental,chili,pepper","live,plant,including,root,ball,ornamental,chil...","[materials, care, live, accessories, animals, ...","[unspsc, root, species, high, variety, live, f...","[pepper, peppers, ornamental, root, live, flow...","[pepper, ornamental, root, live, flowers, chil..."
1998,10000000,"live,plant,animal,material,accessories,supplies","this,segment,includes,live,wild,domesticated,s...",10210000,"live,plants,high,species,variety,count,flowers","live,plants,including,root,ball,flower,types,5...",10216000,"live,ornamental,peppers","live,plants,including,root,ball,ornamental,pep...",10216002,"live,ornamental,mixed,pepper","live,plant,including,root,ball,ornamental,mixe...","[materials, care, live, accessories, animals, ...","[unspsc, root, species, high, variety, live, f...","[pepper, peppers, ornamental, root, live, flow...","[pepper, ornamental, root, live, flowers, ball..."
1999,10000000,"live,plant,animal,material,accessories,supplies","this,segment,includes,live,wild,domesticated,s...",10210000,"live,plants,high,species,variety,count,flowers","live,plants,including,root,ball,flower,types,5...",10216000,"live,ornamental,peppers","live,plants,including,root,ball,ornamental,pep...",10216003,"live,ornamental,orange,pepper","live,plant,including,root,ball,ornamental,oran...","[materials, care, live, accessories, animals, ...","[unspsc, root, species, high, variety, live, f...","[pepper, peppers, ornamental, root, live, flow...","[pepper, ornamental, root, live, flowers, ball..."


In [12]:
unspsc_tc_table_lvl_1 = {}
unspsc_tc_table_lvl_2 = {}
unspsc_tc_table_lvl_3 = {}
unspsc_tc_table_lvl_4 = {}

seg_new_name = 'segment_parsed'
fam_new_name = 'family_parsed'
class_new_name = 'class_parsed'
com_new_name = 'commodity_parsed'

for index, com_code in enumerate(df_csv['Commodity'], 9):
    # if tax_code = NaN
#     if isinstance(seg_code, float):
#         prefix = df_csv["Segment"].loc[index + 1]
#         level_1 = prefix[0:2]
#         level_2 = prefix[2:4]
#         level_3 = "0000"
#         df_xlsx["AvaTax System Tax Code"].loc[index] = prefix[0:4]+"0000"
#     else:
    
    if isinstance(com_code, float):
        continue
    else:
        level_1 = com_code[0:2]+"000000" 
        level_2 = com_code[0:4]+"0000"
        level_3 = com_code[0:6]+"00"
        level_4 = com_code[0:8]


    if level_1 not in unspsc_tc_table_lvl_1:
        unspsc_tc_table_lvl_1[level_1] = df_csv[seg_new_name].loc[index]

    if level_2 not in unspsc_tc_table_lvl_2:
        unspsc_tc_table_lvl_2[level_2] = df_csv[fam_new_name].loc[index]

    if level_3 not in unspsc_tc_table_lvl_3:
        unspsc_tc_table_lvl_3[level_3] = df_csv[class_new_name].loc[index]     

    if level_4 not in unspsc_tc_table_lvl_3:
        unspsc_tc_table_lvl_3[level_3] = df_csv[com_new_name].loc[index]

print(json.dumps(unspsc_tc_table_lvl_1, sort_keys=True, indent=2))
print(json.dumps(unspsc_tc_table_lvl_2, sort_keys=True, indent=2))
print(json.dumps(unspsc_tc_table_lvl_3, sort_keys=True, indent=2))
print(json.dumps(unspsc_tc_table_lvl_4, sort_keys=True, indent=2))

{
  "10000000": [
    "materials",
    "care",
    "live",
    "accessories",
    "animals",
    "plants",
    "plant",
    "containment",
    "this",
    "used",
    "supplies",
    "segment",
    "wild",
    "feeding",
    "seeds",
    "includes",
    "animal",
    "also",
    "equipment",
    "domesticated",
    "material"
  ]
}
{
  "10100000": [
    "animals",
    "live"
  ],
  "10110000": [
    "domestic",
    "pet",
    "products"
  ],
  "10120000": [
    "animal",
    "feed"
  ],
  "10130000": [
    "animal",
    "containment",
    "habitats"
  ],
  "10140000": [
    "saddlery",
    "harness",
    "goods"
  ],
  "10150000": [
    "bulbs",
    "seeds",
    "seedlings",
    "cuttings"
  ],
  "10160000": [
    "silviculture",
    "products",
    "floriculture"
  ],
  "10170000": [
    "nutrients",
    "fertilizers",
    "herbicides",
    "plant"
  ],
  "10190000": [
    "pest",
    "control",
    "products"
  ],
  "10200000": [
    "rose",
    "live",
    "roots",
    "bushes",
   

In [ ]:

# 1.parent node in the avalara file count as a node, do we need to match it to items in the U file
# 2.can we use NLP for matching algo
# 3.as we go down the tree when to stop
# 4.max matching node or 

